#**Kubernetes homework**

In [1]:
!pip install onnx

In [2]:
import os
import shutil
import onnx
import random
import numpy as np
import torch
import torch.nn as nn

from io import BytesIO
from urllib import request
from PIL import Image
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchsummary import summary

In [3]:
!wget https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_v1.onnx.data
!wget https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_v1.onnx

--2025-12-04 09:16:30--  https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_v1.onnx.data
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/426348925/398ded4a-c41c-4e5a-9672-acb7e441de54?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-04T10%3A12%3A25Z&rscd=attachment%3B+filename%3Dhair_classifier_v1.onnx.data&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-12-04T09%3A11%3A42Z&ske=2025-12-04T10%3A12%3A25Z&sks=b&skv=2018-11-09&sig=f79VcEOiul2r13zGDZO%2BUqBhj5ygbU8k%2BCohRmNnFpc%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2NDg0MTU5MCwibmJmIjoxNzY0ODM5Nzk

###**<font color='red'>Question 1</font>**
To be able to use this model, we need to know the name of the input and output nodes.

What's the name of the output:
- <font color='green'>output</font> ✅
- sigmoid
- softmax
- prediction

In [4]:
model = onnx.load("hair_classifier_v1.onnx")
graph = model.graph

print("Inputs:")
for inp in graph.input:
    print("-", inp.name)

print("\nOutputs:")
for out in graph.output:
  print("-", out.name)

Inputs:
- input

Outputs:
- output


###**<font color='red'>Question 2: Target size</font>**
Based on the previous homework, what should be the target size for the image?

- 64x64
- 128x128
- <font color='green'>200x200</font> ✅
- 256x256

###**<font color='red'>Question 3</font>**
Now we need to turn the image into numpy array and pre-process it.

After the pre-processing, what's the value in the first pixel, the R channel?
- -10.73
- <font color='green'>-1.073</font> ✅
- 1.073
- 10.73

In [5]:
def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [6]:
train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

url = "https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg"

img = download_image(url)
img = prepare_image(img, (200, 200))
tensor = train_transforms(img)
array = tensor.numpy()
print("First R pixel:", array[0, 0, 0])

First R pixel: -1.073294
